### Goal is to look into the large scale tokenization of the entire dataset


#### Takeaways: We CAN TF-IDF 2 million, but memory is a big issue? That on top of not playing around with embeddings yet is a slightly sussy thought.

In [ ]:
import pandas as pd
import numpy as np
import pickle
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset
from datasets import load_dataset
import datasets
import torch
from collections import Counter
import string
from scipy import spatial
import nltk
from nltk.stem import RegexpStemmer

In [ ]:
dataset_dict = datasets.load_from_disk("./Dataset/CodeSearchCorpus/")
train_dataset = dataset_dict["train"]

In [ ]:
# train_dataset.to_pandas() Not feasible, 90% memory

In [ ]:
# train_dataset

In [ ]:
# num_rows = 50000
# np.random.seed(1)
# train_subset_indices = np.random.choice(len(train_dataset), num_rows, replace = False)
# train_dataset_subset = train_dataset.select(train_subset_indices)
# len(train_dataset_subset)

In [ ]:
# train_dataset_subset

In [ ]:
# train_dataset_subset.select_columns("func_documentation_string")

In [ ]:
# train_dataset_subset = train_dataset_subset.select_columns("func_documentation_string")

In [ ]:
# train_dataset_subset.to_pandas()

In [ ]:
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer

st = RegexpStemmer('ing$|s$|e$|able$', min=4)
tokenizer = RegexpTokenizer(r'\w+')
# tokenizer.tokenize('Eighty-seven miles to go, yet.  Onward!')

In [ ]:
train_dataset = train_dataset.select_columns("func_documentation_string")

# train_dataset

In [ ]:
# train_dataset = train_dataset.map(lambda tkn: {"func_doc_tokens": tokenizer.tokenize(tkn)})

In [ ]:
train_dataset_DF = train_dataset.to_pandas()

In [ ]:
# tokenizer.tokenize(test_sentence)

In [ ]:
train_dataset_DF["func_doc_tokens"] = train_dataset_DF["func_documentation_string"].apply(lambda x: tokenizer.tokenize(x))

#25.3 seconds

In [ ]:
train_dataset_DF["func_doc_stem_tokens"] = train_dataset_DF["func_doc_tokens"].apply(lambda x: [st.stem(word) for word in x])
# ~35 seconds

In [ ]:
# test_tokens
# [ps.stem(word) for word in test_tokens]

In [ ]:
inverted_index = {}
for i in range(len(train_dataset_DF)):
    token_counter = Counter(train_dataset_DF.iloc[i]["func_doc_stem_tokens"])

    for token in token_counter:
        if token not in inverted_index:
            inverted_index[token] = {}
        inverted_index[token][i] = token_counter[token]

#Took ~2 minutes

In [ ]:
# len(inverted_index)

In [ ]:
#73622 No stemming
#68995 with stemming
#Pickle afterwards
with open("./pickleObjects/inverted_index_FULL.pkl", 'wb') as f:  # open a text file
    pickle.dump(inverted_index, f) # serialize the list
    f.close()

In [ ]:
# with open("./pickleObjects/tfidf_FULL.pkl", 'wb') as f:  # open a text file
#     pickle.dump(tf_idf, f) # serialize the list
#     f.close()

In [ ]:
with open('./pickleObjects/inverted_index_FULL.pkl', 'rb') as f:
    inverted_index = pickle.load(f) # deserialize using load()
    f.close()

In [ ]:
tf_idf = {}
for i in range(len(train_dataset_DF)):
    # print(i)
    tokens = train_dataset_DF["func_doc_stem_tokens"].iloc[i]
    counter = Counter(tokens)
    words_count = len(tokens)

    for token in np.unique(tokens):
        tf = counter[token] / words_count
        df = len(inverted_index[token])
        idf = np.log((len(train_dataset_DF) + 1) / (df + 1))

        tf_idf[i, token] = tf * idf

# 4 minutes there or thereabouts

In [ ]:
# with open("./pickleObjects/tfidf_FULL.pkl", 'wb') as f:  # open a text file
#     pickle.dump(tf_idf, f) # serialize the list
#     f.close()

In [ ]:
# with open('./pickleObjects/tfidf_FULL.pkl', 'rb') as f:
#     tf_idf = pickle.load(f) # deserialize using load()
#     f.close()

In [ ]:
def query_results(query_string, inverted_index, tf_idf, k = 10):
    query_tokens = [st.stem(word) for word in tokenizer.tokenize(query_string)]
    print(query_tokens)
    rel_indices = []
    
    for token in query_tokens:
        if token in inverted_index:
            rel_indices += list(inverted_index[token].keys())
    
    rel_indices = set(rel_indices)

    # len(query_embedding[0])
    # len(tsed_DF["embeddings"][0])
    

    result_lst = []
    for i in rel_indices:
        for token in query_tokens:
            tf_score = 0
            try:
                # print(tf_idf[(i, token)])
                tf_score += (tf_idf[(i, token)])
            except: 
                continue #this is bad, make sure this isn't the play
        # print(i)

        result_lst.append([i, tf_score])
    
    result_lst.sort(reverse=True, key = lambda x: x[1])
    return result_lst[:k]

In [ ]:
query_results("string to datetime", inverted_index, tf_idf, k = 50)

In [ ]:
train_dataset[100930]

In [ ]:
with open("pickleObjects/train_subset_embeddings_dataset_100000_cb.pkl", 'rb') as f:  # open a text file
            embed_dataset = pickle.load(f) # serialize the list
            f.close()

In [ ]:
embed_dataset